检验矩阵是否是正互反矩阵。矩阵为方阵，对角线为1，对称位置上的元素互为倒数。

In [2]:
import numpy as np


def is_reciprocal(matrix):
    if matrix.ndim != 2:
        return False
    
    if matrix.shape[0] != matrix.shape[1]:
        return False
    
    if not all([matrix[i][j] * matrix[j][i] == 1 for i in range(matrix.shape[0]) for j in range(matrix.shape[1])]):
        return False
    
    return True


m1 = np.array([[1, 3, 5], [1/3, 1, 2], [1/5, 1/2, 1]])
print(is_reciprocal(m1))

m2 = np.array([2, 3, 1, 5, 3, 7]).reshape(3, -1)
print(is_reciprocal(m2))

True
False


判断正互反矩阵是否满足一致性要求：每一行(列)的元素之间的比例都相同。从统计学意义上来说，当CR=CI/RI<0.1时，认为满足一致性要求。其中CI=({\lambda}_
{max}-n)/(n-1)，RI和n有关，可以通过查表得到。其中{/lambda}_{max}是矩阵的最大特征值。


In [3]:
def get_max_eigenvalue(matrix):
    eigen_values = np.linalg.eigvals(matrix)
    return np.max(eigen_values)


def is_consistent(matrix):
    max_eigenvalue = get_max_eigenvalue(matrix)
    n = matrix.shape[0]
    
    ci = (max_eigenvalue - n) / (n - 1)
    ri = get_ri(n)
    cr = ci / ri
    
    return cr < 0.1


def get_ri(n):
    table = {
        1: 0, 
        2: 0, 
        3: 0.58,
        4: 0.9,
        5: 1.12,
        6: 1.24,
        7: 1.32,
        8: 1.41,
        9: 1.45,
        10: 1.49
    }
    
    return table[n]
    

m1 = np.array([[1, 3, 5], [1/3, 1, 2], [1/5, 1/2, 1]])
m2 = np.array([1, 3, 2, 3, 9, 6, 2, 6, 4]).reshape(3, 3)
print(is_consistent(m1))
print(is_consistent(m2))

True
False


如果矩阵符合一致性，那么我们可以使用矩阵的最大特征值对应的特征向量的第一列作为权重。

In [9]:
def get_weights(matrix):
    # 计算特征值和特征向量。
    eigen_values, eigenvectors = np.linalg.eig(matrix)
    
    # 找到最大特征值的索引。
    max_eigenvalue_index = np.argmax(eigen_values)
    
    # 最大特征值对应的特征向量。
    max_eigenvector = eigenvectors[:, max_eigenvalue_index]
    
    weights = [max_eigenvector[i] / sum(max_eigenvector) for i in range(matrix.shape[0])]
    
    return weights
    
    
m1 = np.array([[1, 3, 5], [1/3, 1, 2], [1/5, 1/2, 1]])
print(get_weights(m1))
    

[np.complex128(0.6483290138222366+0j),
 np.complex128(0.2296507940626371+0j),
 np.complex128(0.12202019211512623+0j)]

如果矩阵不符合一致性，我们需要得到每一列的权重，然后进行算数平均，得到最终的权重。

In [10]:
def get_weights(matrix):
    weights = [np.sum(matrix[i, :]) / np.sum(matrix) for i in range(matrix.shape[1])]
    return weights


m1 = np.array([[1, 3, 5], [1/3, 1, 2], [1/5, 1/2, 1]])
print(get_weights(m1))

[np.float64(0.641330166270784),
 np.float64(0.23752969121140144),
 np.float64(0.12114014251781474)]

给定第一列的元素，生成正互反矩阵。

In [8]:
def generate_reciprocal_matrix(column):
    n = len(column)
    matrix = np.array([1.0 for i in range(n*n)]).reshape(n, n)
    ratio = [column[i] / column[i + 1] for i in range(n - 1)]
    count = 0
    
    while count < n:
        for i in range(count + 1, n):
            matrix[i][count] = matrix[i - 1][count] / ratio[i - 1]
            
        for i in range(count - 1, -1, -1):
            matrix[i][count] = matrix[i + 1][count] * ratio[i]
        
        count += 1
    
    return matrix


print(generate_reciprocal_matrix([1, 2, 0.4]))
    

[[1.  0.5 2.5]
 [2.  1.  5. ]
 [0.4 0.2 1. ]]


综合过程：列出一个表格，根据直觉完成第一列的不同属性的分配。调用generate_reciprocal_matrix函数，生成正互反矩阵。判断正互反矩阵的一致性，根据一致性的异同执行相应的get_weights函数，得到权重。